<a href="https://colab.research.google.com/github/Priya1881/NLP/blob/main/NewsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.layers.embeddings import Embedding



In [ ]:
def read_files(path):
    file_contents=[]
    filenames=os.listdir(path)
    for i in range(len(filenames)):
        with open(path + filenames[i],encoding="ISO-8859-1") as f:
            file_contents.append(f.read())
    return file_contents
    

In [ ]:
class_0 = read_files('/content/drive/MyDrive/comp.graphics/')
class_1 = read_files('/content/drive/MyDrive/rec.motorcycles/')
class_2 = read_files('/content/drive/MyDrive/sci.med/')
class_3 = read_files('/content/drive/MyDrive/talk.politics.misc/')



In [ ]:
all_texts=np.append(class_0,class_1)
all_texts=np.append(all_texts,class_2)
all_texts= np.append(all_texts,class_3)

In [ ]:
import nltk
nltk.download('stopwords')


In [ ]:
stop_words= set(stopwords.words('english'))

def clean(text):
    #lowering letters
    text = text.lower()
    #removing html tags
    text = re.sub('<[^>]*>',' ',text)
    # removing emails
    text = re.sub('\S*@\S*\s?',' ',text)
    # removing urls
    text = re.sub('https?://[A-Za-z0-9]',' ',text)
    #removing numbers
    text = re.sub('[^A-Za-z]',' ',text)
    
    word_tokens=word_tokenize(text)
    filtered_sentence=[]
    for word_token in word_tokens:
        if word_token not in stop_words and len(word_token) > 2:
            filtered_sentence.append(word_token)
    #joining words
    text=' '.join(filtered_sentence)
    return text

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
all_cleaned_texts = np.array([clean(text) for text in all_texts])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_cleaned_texts)
all_encoded_texts=tokenizer.texts_to_sequences(all_cleaned_texts)

In [ ]:
all_encoded_texts = np.array(all_encoded_texts)
len(all_encoded_texts)

In [ ]:
all_encoded_texts = sequence.pad_sequences(all_encoded_texts, maxlen=500)



In [ ]:
labels_0=np.array([0]*len(class_0))
labels_1=np.array([1]*len(class_1))
labels_2=np.array([2]*len(class_2))
labels_3=np.array([3]*len(class_3))

In [ ]:
all_labels = np.append(labels_0,labels_1)
all_labels = np.append(all_labels,labels_2)
all_labels = np.append(all_labels,labels_3)

In [ ]:
all_labels=all_labels[:,np.newaxis]

In [ ]:
one_hot_encoder=OneHotEncoder(sparse=False)
all_labels=one_hot_encoder.fit_transform(all_labels)
print(all_labels.shape)

In [ ]:
Xtrain,Xtest,ytrain,ytest=train_test_split(all_encoded_texts,all_labels,test_size=0.2,random_state=11)

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=34869,output_dim=32,input_length=500))
model.add(LSTM(100))
model.add(Dense(4,activation='sigmoid'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])



In [ ]:
history=model.fit(Xtrain,ytrain,epochs=12,batch_size=64,validation_data=(Xtest,ytest))

In [ ]:
predictions = model.predict(Xtest)
predictions = one_hot_encoder.inverse_transform(predictions)

In [ ]:
ytest_evaluate = np.argmax(ytest,axis=1)

In [ ]:
cm=confusion_matrix(ytest_evaluate,predictions)

In [ ]:
cm

In [ ]:
string='''Yes, very serious.  I claim that I can substantiate my statement that
  Rudman says he doesn't believe Perot was investigating him.  You claim
Perot was investigating him.  If you will state that you were in error
on this point, provided I produce the source, I'll go dig it up.'''

In [ ]:
cleaned_string = clean(string)

In [ ]:
cleaned_string

In [ ]:
encoded_string = tokenizer.texts_to_sequences([cleaned_string])

In [ ]:
encoded_string = sequence.pad_sequences(encoded_string,maxlen=500)

In [ ]:
string_predicted = model.predict(encoded_string)
np.argmax(string_predicted)